In [4]:
%pip -q install google-genai

In [5]:
import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [6]:
from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [7]:
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/

In [8]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types
from datetime import date
import textwrap
from IPython.display import display, Markdown
import requests
import warnings

warnings.filterwarnings("ignore")

In [9]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [10]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [11]:
##########################################
# --- Agente 1: Fará uma pesquisa aprofundada sobre o uso de celular.
##########################################
def agente_pesquisador(form_user):

    agente_pesquisador_obj = Agent( # Renomeada a variável do objeto Agent para evitar conflito com o nome da função
        name="agente_pesquisador",
        model="gemini-2.0-flash",
        instruction="""
        Sua missão é construir uma base de conhecimento sólida e abrangente sobre o uso de telefones celulares e seus impactos com base em sua pesquisa no Google (google_search). Realize uma pesquisa detalhada focando nos seguintes aspectos:
        Recomendações de Especialistas: Investigue diretrizes médicas e de especialistas sobre o tempo ideal de uso para diversas faixas etárias, identificando os riscos associados ao uso excessivo (saúde física, mental, qualidade do sono, etc.).
        Panorama do Uso Atual: Colete dados estatísticos sobre a média de tempo de uso de celulares globalmente e, especificamente, no Brasil. Busque por variações demográficas relevantes (idade, profissão, etc.).
        Estratégias de Redução Eficazes: Descubra e catalogue métodos e técnicas comprovadas para diminuir o tempo de tela, como aplicativos de monitoramento, agendamento de tempo de uso, desativação de notificações e outras abordagens práticas.
        Hobbies e Atividades Alternativas: Reúna uma lista diversificada e inspiradora de hobbies e atividades offline que possam substituir o tempo gasto no celular (exercícios físicos, leitura, atividades artísticas, interação social, aprendizado, etc.).
        Impacto Psicológico e Social: Pesquise sobre os efeitos do uso excessivo de celular na saúde mental (ansiedade, depressão, comparação social, etc.) e na qualidade das relações interpessoais.
        Fontes de Informação: Priorize informações provenientes de fontes confiáveis, como estudos científicos publicados, artigos de especialistas reconhecidos, organizações de saúde renomadas e pesquisas de instituições acadêmicas.
        Seu objetivo é entregar um conjunto rico e bem referenciado de informações que servirá como matéria-prima essencial para os próximos agentes.
        Limite sua pesquisa às últimas notícias e pesquisas relevantes até 5 fontes.
        """,
        description="Agente que busca informações no Google",
        tools=[google_search]
    )

    entrada_do_agente_pesquisador = f"Se baseie nos dados do usuários para fazer sua pesquisa: {form_user}"

    # Chame call_agent com o objeto Agent e a mensagem de entrada
    formulario_result = call_agent(agente_pesquisador_obj, entrada_do_agente_pesquisador)
    # Retorne o resultado da chamada, não a variável 'lancamentos' que não está definida aqui
    return formulario_result

In [17]:
################################################
# --- Agente 2: Seletor de dados --- #
################################################
def Seletor_Dados(form_user, formulario):
    seletor = Agent(
        name="agente_seletor",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
            Sua tarefa é refinar e direcionar as informações brutas fornecidas pesquisadas, transformando-as em um conjunto conciso e aplicável para ajudar um usuário a regular o uso do celular.
            Análise Crítica: Avalie cuidadosamente todas as informações recebidas, identificando os dados mais impactantes e relevantes para o objetivo de reduzir o tempo de tela.
            Foco na Praticidade: Selecione informações que possam ser facilmente compreendidas e implementadas pelo usuário em sua vida diária.
            Extração de Pontos Chave: Identifique as principais conclusões, recomendações diretas e estatísticas mais significativas que emergem da pesquisa.
            Eliminação de Redundância: Remova informações repetitivas ou que apresentem o mesmo conceito de maneira muito similar.
            Organização Temática: Agrupe as informações selecionadas em categorias lógicas (por exemplo, recomendações de saúde, estratégias práticas, sugestões de hobbies) para facilitar o uso pelo Agente 3.
            Você também pode usar o (google_search) para refinar ainda mais esses resultados.
            Justificativa (Opcional, mas Recomendado): Se possível, inclua uma breve explicação para cada dado selecionado, indicando por que você o considerou particularmente relevante para o usuário.
            Seu objetivo é entregar um conjunto de informações filtradas e bem organizadas que formará a espinha dorsal do relatório personalizado para o usuário.
        """,
        description="Agente que refina a pesquisa feita e relaciona com dados fornecidos pelo usuário",
        tools=[google_search]
    )

    seletor_result = f"Formulário do usuário é:{form_user}\nLançamentos buscados: {formulario}"
    # Executa o agente
    filtroPesquisa = call_agent(seletor, seletor_result)
    return filtroPesquisa

In [13]:
######################################
# --- Agente 3: Relatório geral com dados do Usuário --- #
######################################
def relatorio(form_user, filtroPesquisa):
    agente_relatorio = Agent(
        name="agente_relator",
        model="gemini-2.0-flash",
        instruction="""
            Utilizando os dados selecionados pelo Agente seletor e as informações fornecidas sobre o usuário (form_user), sua tarefa é criar um relatório personalizado e motivador para ajudar o usuário a regular o tempo de uso do celular.
            Adapte as informações: Conecte os dados gerais sobre o uso de celular com a situação e os objetivos específicos do usuário. Por exemplo, se o usuário mencionou dificuldades para dormir, destaque as informações sobre o impacto do celular no sono.
            Estrutura clara e lógica: Organize o relatório em seções fáceis de entender, como:
            Introdução: Um resumo do problema do uso excessivo de celular e a importância de encontrar um equilíbrio.
            Dados relevantes: Apresente estatísticas e recomendações médicas que ressoem com a situação do usuário.
            Estratégias personalizadas: Sugira estratégias práticas para reduzir o uso, levando em consideração a rotina e preferências do usuário.
            Alternativas de hobbies: Ofereça uma lista de hobbies e atividades offline que possam interessar ao usuário, com base em seus possíveis interesses (se fornecidos) ou sugestões gerais.
            Benefícios de reduzir o uso: Destaque os ganhos em termos de saúde, bem-estar e qualidade de vida.
            Mensagem de encorajamento: Finalize com uma mensagem positiva e motivadora para apoiar o usuário em sua jornada.
            Linguagem clara e acessível: Utilize uma linguagem amigável, evitando jargões técnicos.
            Tom motivacional: Adote um tom de apoio e incentivo, visando inspirar o usuário a fazer mudanças positivas.
            Seu objetivo é transformar dados brutos em um guia prático e inspirador que motive o usuário a alcançar um uso mais consciente e saudável do celular.
            """,
        description="Agente que criará um relatório com base nos dados do usuário e a pesquisa"
    )
    relatorio_usuario = f"Formulário do usuário é:{form_user}\nPesquisa filtrada: {Seletor_Dados}"
    # Executa o agente
    rascunho_relatorio = call_agent(agente_relatorio, relatorio_usuario)
    return rascunho_relatorio

In [14]:
##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, rascunho_relatorio):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash",
        instruction="""
            Sua tarefa final é revisar cuidadosamente o relatório gerado pelo rascunho do relatorio para garantir a correção gramatical, ortográfica e de concordância.
            Se o conteúdo estiver muito extenso, faça um resumo para que fique melhor para o usuário visualizar
            Se julgar necessário, pode fazer um pesuisa no (google_search) para auxiliar na revisão e ver se os dados ainda batem.
            """,
        description = "Agente revisor de relatório.",
        tools=[google_search]
    )
    entrada_do_agente_revisor = f"Formulário do usuário é:{form_user}\nRascunho do Relatório: {rascunho_relatorio}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [15]:
form_user = input("Digite aqui:")

formulario = agente_pesquisador(form_user)

filtroPesquisa = Seletor_Dados(form_user, formulario)

rascunho_relatorio = relatorio(form_user, filtroPesquisa)

texto_revisado = agente_revisor(form_user, rascunho_relatorio)

print(rascunho_relatorio)
display(to_markdown(rascunho_relatorio))

Digite aqui:


NameError: name 'pesquisa' is not defined

In [18]:
print("Olá, eu sou o Concentra.AI, seu assistente inteligente dedicado a\nte ajudar a retomar o controle do seu tempo e a cultivar um relacionamento mais saudável com o seu celular\nEm um mundo cada vez mais conectado, é fácil se sentir sobrecarregado e distraído pelo fluxo constante de notificações\ne informações. Eu estou aqui para te guiar em uma jornada de uso mais consciente\ne equilibrado do seu dispositivo.")

# --- Obter o Tópico do Usuário ---
form_user = [input("\nEm média, quantas horas por dia\nvocê estima que passa usando o celular?: "),
             input("\nQuais são os principais momentos ou situações em que\nvocê sente que usa o celular excessivamente?: "),
             input("\nQuais aplicativos ou atividades você mais utiliza no celular?"),
             input("\nVocê já tentou reduzir o tempo\nde uso do celular antes?: "),
             input("\nSe sim, quais foram os resultados?\nO que te ajudou ou dificultou o processo?"),
             input("\nQuais são seus principais objetivos ao tentar reduzir\no tempo de uso do celular? O que você gostaria de fazer ou sentir mais ao diminuir o tempo de tela?: "),
             input("\nQuais são seus principais hobbies ou interesses em seu\ntempo livre (fora do celular)?: ")]

# Inserir lógica do sistema de agentes ################################################
if not form_user:
    print("Você esqueceu de digitar os dados necessários para que a gente consiga te ajudar. Tente novamente.")
else:
    print(f"\nObrigado por suas respostas! Com base nessas informações, o Concentra.AI\nirá gerar um relatório personalizado para te auxiliar\nem sua jornada para um uso mais consciente\ne equilibrado do seu celular.")

    formulario = agente_pesquisador(form_user)
    print("\n--- Vou fazer uma pesquisa sobre o assunto ---\n")
    display(to_markdown(formulario))
    print("--------------------------------------------------------------")

    filtroPesquisa = Seletor_Dados(form_user, formulario)
    print("\n--- Vou refinar essa pesquisa e trazer somente os dados mais relevantes ---\n")
    display(to_markdown(filtroPesquisa))
    print("--------------------------------------------------------------")

    rascunho_relatorio = relatorio(form_user, filtroPesquisa)
    print("\n--- Agora vou pensar em um relatório com base nos dados que você me forneceu ---\n")
    display(to_markdown(rascunho_relatorio))
    print("--------------------------------------------------------------")

    texto_revisado = agente_revisor(form_user, rascunho_relatorio)
    print("\n--- Aqui está seu relatório, espero que te ajude ---\n")
    display(to_markdown(texto_revisado))
    print("--------------------------------------------------------------")


Olá, eu sou o Concentra.AI, seu assistente inteligente dedicado a
te ajudar a retomar o controle do seu tempo e a cultivar um relacionamento mais saudável com o seu celular
Em um mundo cada vez mais conectado, é fácil se sentir sobrecarregado e distraído pelo fluxo constante de notificações
e informações. Eu estou aqui para te guiar em uma jornada de uso mais consciente
e equilibrado do seu dispositivo.

Em média, quantas horas por dia
você estima que passa usando o celular?: 5

Quais são os principais momentos ou situações em que
você sente que usa o celular excessivamente?: Em casa

Quais aplicativos ou atividades você mais utiliza no celular?Yotuube

Você já tentou reduzir o tempo
de uso do celular antes?: Não

Se sim, quais foram os resultados?
O que te ajudou ou dificultou o processo?Não

Quais são seus principais objetivos ao tentar reduzir
o tempo de uso do celular? O que você gostaria de fazer ou sentir mais ao diminuir o tempo de tela?: Otimizar meu tempo

Quais são seus princ

> Com base nas informações fornecidas, vou realizar uma pesquisa focada nos seguintes pontos, considerando a idade do usuário (5 anos), o local de uso predominante (em casa), o aplicativo mais utilizado (YouTube), a ausência de estratégias de otimização de tempo e a prática de andar de skate como atividade alternativa:
> 
> 1.  **Recomendações de especialistas para tempo de tela para crianças de 5 anos:** Quais são as diretrizes médicas e de especialistas sobre o tempo ideal de uso de tela para crianças de 5 anos, com foco em YouTube e uso em casa? Quais os riscos associados ao uso excessivo em termos de saúde física, mental e sono?
> 2.  **Estratégias de redução de tempo de tela adequadas para crianças de 5 anos:** Quais métodos e técnicas comprovadas são mais eficazes para diminuir o tempo de tela em crianças de 5 anos, considerando o uso predominante em casa e no YouTube? Como envolver os pais nesse processo?
> 3.  **Atividades alternativas para crianças de 5 anos além de andar de skate:** Que outros hobbies e atividades offline são recomendadas para crianças de 5 anos, além de andar de skate, para promover o desenvolvimento físico, cognitivo e social?
> 4.  **Impacto do uso excessivo de YouTube em crianças:** Quais são os efeitos do uso excessivo do YouTube na saúde mental (ansiedade, atenção, etc.) e no desenvolvimento social de crianças de 5 anos?
> 5.  **Fontes de informação confiáveis sobre tempo de tela e desenvolvimento infantil:** Quais são as fontes de informação mais confiáveis (estudos científicos, artigos de especialistas, organizações de saúde) sobre o impacto do tempo de tela e estratégias de redução para crianças de 5 anos?
> 
> 
> Com base nas informações encontradas, aqui está um resumo detalhado e estruturado para auxiliar o usuário:
> 
> **1. Recomendações de especialistas para tempo de tela para crianças de 5 anos:**
> 
> *   **Tempo recomendado:** A Sociedade Brasileira de Pediatria (SBP), a Academia Americana de Pediatria (AAP) e a Organização Mundial da Saúde (OMS) recomendam **no máximo 1 hora por dia** para crianças menores de 5 anos. É crucial que esse tempo seja supervisionado por um adulto.
> *   **Riscos do uso excessivo:**
>     *   **Saúde física:** Estilo de vida sedentário, levando à obesidade e problemas de saúde relacionados. Problemas posturais e dores musculares também podem surgir.
>     *   **Sono:** Dificuldade para dormir, interferência na qualidade e duração do sono, podendo levar à sonolência diurna.
>     *   **Desenvolvimento cognitivo:** Problemas de concentração, dificuldades de interação e adaptação ao meio social, diminuição da capacidade de comunicação e resolução de problemas.
>     *   **Saúde mental:** Irritabilidade, ansiedade, depressão, baixa autoestima e, em adolescentes, transtornos da imagem corporal. Comparação constante com os outros nas redes sociais pode exacerbar esses problemas.
>     *   **Dependência tecnológica:** Uso compulsivo, irritação ou ansiedade quando afastados dos dispositivos.
> 
> **2. Estratégias de redução de tempo de tela adequadas para crianças de 5 anos:**
> 
> *   **Estabelecer limites claros:** Definir um tempo máximo diário de uso e horários específicos.
> *   **Supervisão:** Usar as telas sempre com supervisão de pais, cuidadores ou responsáveis.
> *   **Conteúdo de qualidade:** Selecionar conteúdos apropriados e de qualidade, focados em aprendizado e desenvolvimento. Atuar como "curadores digitais", escolhendo o que é realmente benéfico.
> *   **Zonas e horários livres de tela:** Criar zonas livres de telas em casa, como durante as refeições e antes de dormir. Desconectar-se 1 a 2 horas antes de dormir.
> *   **Atividades alternativas:**
>     *   Incentivar atividades físicas e ao ar livre, como andar de skate, jogos, esportes, e brincadeiras que estimulem a interação com outras crianças.
>     *   Promover vínculos afetivos e minimizar os efeitos negativos do excesso de exposição às telas.
>     *   Oferecer outras formas de entretenimento, como leitura, atividades artísticas (pintura com as mãos, teatro de fantoches), caça ao tesouro, jogos de quebra-cabeças, dança, e outras brincadeiras.
>     *   Incentivar a participação em jogos de equipe, esportes e atividades que estimulem a interação com outras crianças para construir habilidades como empatia, cooperação e resiliência.
> *   **Envolvimento dos pais:** Liderar pelo exemplo, modelando hábitos saudáveis de mídia para os filhos. Participar da visualização compartilhada.
> *   **Comunicação:** Promover a comunicação aberta e explicar a importância do equilíbrio e da moderação.
> 
> **3. Atividades alternativas para crianças de 5 anos além de andar de skate:**
> 
> *   **Atividades físicas:** Pega-pega, pular corda, cambalhotas, corridas, saltos de baixa intensidade, ginástica olímpica, natação, danças, artes marciais de baixo impacto.
> *   **Brincadeiras ao ar livre:** Amarelinha, esconde-esconde, jogos com bola.
> *   **Atividades criativas:** Pintura com as mãos, teatro de fantoches, caça ao tesouro.
> *   **Jogos:** Jogos de quebra-cabeças, dança das cadeiras.
> *   **Outras atividades:** Contato com a natureza, leitura, atividades que envolvam a família.
> 
> **4. Impacto do uso excessivo de YouTube em crianças:**
> 
> *   **Problemas de saúde mental:** Depressão, ansiedade, baixa autoestima e transtornos de imagem corporal.
> *   **Problemas cognitivos:** Pode prejudicar o desenvolvimento cognitivo, afetando a capacidade de concentração, memória e habilidades de resolução de problemas.
> *   **Dificuldades sociais:** Dificuldades de interação e adaptação ao meio social.
> *   **Exposição a conteúdos inadequados:** Risco de exposição a conteúdos violentos, sexuais ou perturbadores.
> 
> **5. Fontes de informação confiáveis sobre tempo de tela e desenvolvimento infantil:**
> 
> *   Sociedade Brasileira de Pediatria (SBP).
> *   Academia Americana de Pediatria (AAP).
> *   Organização Mundial da Saúde (OMS).
> *   Artigos científicos e estudos de universidades e instituições de pesquisa.
> *   Manuais de orientação sobre saúde na era digital.
> 
> É importante lembrar que o equilíbrio é fundamental. O uso de telas pode ter benefícios, como acesso à informação e entretenimento educativo, mas o excesso pode ser prejudicial. A chave é a moderação, a supervisão e o oferecimento de alternativas saudáveis e estimulantes.
> 


--------------------------------------------------------------

--- Vou refinar essa pesquisa e trazer somente os dados mais relevantes ---



> Com base nas informações fornecidas e no perfil do usuário, selecionei e organizei os dados mais relevantes para ajudá-lo a regular o uso do celular de seu filho de 5 anos:
> 
> **1. Recomendações de especialistas sobre tempo de tela:**
> 
> *   **Tempo Máximo:** Limitar o uso a no máximo 1 hora por dia, sempre com supervisão. Isso é crucial para a saúde física, mental e cognitiva da criança.
> *   **Riscos:**
>     *   Problemas de sono: Dificuldade para dormir e má qualidade do sono.
>     *   Desenvolvimento: Problemas de concentração, interação social e comunicação.
>     *   Saúde Mental: Irritabilidade e ansiedade.
>     *   Sedentarismo: Aumento do risco de obesidade.
> 
> **2. Estratégias práticas para reduzir o tempo de tela:**
> 
> *   **Limites Claros:** Defina horários específicos e tempo máximo de uso diário.
> *   **Supervisão Contínua:** Garanta que o uso das telas seja sempre supervisionado.
> *   **Conteúdo de Qualidade:** Escolha vídeos e aplicativos educativos e apropriados para a idade.
> *   **Zonas Livres de Tela:** Evite telas durante as refeições e 1-2 horas antes de dormir.
> *   **Atividades Alternativas:**
>     *   Incentive atividades físicas como andar de skate, esportes e brincadeiras ao ar livre.
>     *   Promova atividades criativas como pintura, teatro de fantoches e leitura.
>     *   Estimule jogos de quebra-cabeças e brincadeiras que envolvam a família.
> *   **Exemplo dos Pais:** Mostre hábitos saudáveis de uso de mídia.
> *   **Comunicação Aberta:** Explique a importância do equilíbrio e da moderação.
> 
> **3. Atividades alternativas (além de andar de skate):**
> 
> *   **Físicas:** Pega-pega, pular corda, natação, dança, artes marciais de baixo impacto.
> *   **Ao ar livre:** Amarelinha, esconde-esconde, jogos com bola.
> *   **Criativas:** Pintura com as mãos, teatro de fantoches, caça ao tesouro.
> *   **Jogos:** Quebra-cabeças, dança das cadeiras.
> *   **Outras:** Contato com a natureza, leitura em família.
> 
> **4. Impacto específico do uso excessivo de YouTube:**
> 
> *   **Saúde Mental:** Risco de depressão, ansiedade e baixa autoestima.
> *   **Cognitivo:** Prejuízo na concentração, memória e resolução de problemas.
> *   **Social:** Dificuldades de interação social.
> *   **Conteúdo Inadequado:** Exposição a conteúdos violentos ou perturbadores.
> 
> **5. Fontes confiáveis para mais informações:**
> 
> *   Sociedade Brasileira de Pediatria (SBP)
> *   Academia Americana de Pediatria (AAP)
> *   Organização Mundial da Saúde (OMS)
> 
> Lembre-se: O objetivo é o equilíbrio. As telas podem ser benéficas, mas o excesso é prejudicial. Moderação, supervisão e alternativas saudáveis são a chave.
> 


--------------------------------------------------------------

--- Agora vou pensar em um relatório com base nos dados que você me forneceu ---



> Olá! Entendo que você está buscando otimizar seu tempo e reduzir o uso do celular, especialmente em casa e com foco no YouTube. Que ótimo que você já identificou isso e está tomando uma atitude! Abaixo, preparei um relatório com algumas informações e dicas que podem te ajudar nessa jornada.
> 
> **Introdução:**
> 
> Hoje em dia, é super comum passarmos mais tempo do que gostaríamos no celular. O problema é que esse uso excessivo pode nos impedir de aproveitar outras coisas importantes na vida, como nossos hobbies, amigos e até mesmo nossa saúde. Mas não se preocupe, com algumas estratégias e um pouco de disciplina, é possível encontrar um equilíbrio saudável!
> 
> **Dados Relevantes:**
> 
> *   **Tempo médio:** Usuários passam em média 5 horas por dia no celular.
> *   **YouTube:** O YouTube é uma das plataformas que mais prendem a atenção, com vídeos que muitas vezes são feitos para serem viciantes.
> *   **Impacto em casa:** Usar muito o celular em casa pode diminuir o tempo que você passa com a família, fazendo atividades físicas ou relaxando de verdade.
> *   **Recomendação:** Especialistas recomendam limitar o uso diário do celular a no máximo 2 horas para um bem estar saudável.
> 
> **Estratégias Personalizadas:**
> 
> 1.  **Defina horários:** Já que você usa mais o celular em casa, que tal estabelecer horários específicos para checar o YouTube? Por exemplo, 30 minutos pela manhã e 30 minutos à noite.
> 2.  **Aplicativos:** Existem aplicativos que monitoram o tempo de uso e até bloqueiam o acesso a determinados apps depois de um tempo. Experimente alguns para ver qual funciona melhor para você.
> 3.  **Notificações:** Desative as notificações do YouTube. Assim, você evita ser interrompido a todo momento e perde menos tempo abrindo o aplicativo por impulso.
> 4.  **Substituição:** Quando sentir vontade de pegar o celular, tente fazer outra coisa que goste. Que tal pegar seu skate e dar uma volta?
> 
> **Alternativas de Hobbies:**
> 
> *   **Andar de Skate:** Você já tem um ótimo hobby! Que tal explorar novos lugares para andar de skate, aprender manobras novas ou até fazer um vídeo para compartilhar com outros skatistas?
> *   **Esportes:** Além de skate, experimente outros esportes como corrida, bicicleta ou natação.
> *   **Leitura:** Descubra novos livros ou revistas sobre assuntos que te interessam.
> *   **Música:** Aprenda a tocar um instrumento, cante, dance ou simplesmente relaxe ouvindo suas músicas favoritas.
> *   **Cozinhar:** Experimente novas receitas, convide amigos para jantar ou faça um curso de culinária.
> 
> **Benefícios de Reduzir o Uso:**
> 
> *   **Mais tempo:** Você terá mais tempo para se dedicar a seus hobbies e outras atividades que te dão prazer.
> *   **Melhor foco:** Diminuir o uso do celular ajuda a melhorar a concentração e a produtividade.
> *   **Menos ansiedade:** Reduzir a exposição constante a notícias e redes sociais diminui a ansiedade e o estresse.
> *   **Melhor sono:** Evitar o celular antes de dormir melhora a qualidade do sono.
> *   **Mais conexão:** Você estará mais presente nos momentos com amigos e familiares.
> 
> **Mensagem de Encorajamento:**
> 
> Lembre-se que mudar hábitos leva tempo e esforço. Não se cobre demais se não conseguir de primeira. O importante é persistir e celebrar cada pequena vitória. Acredite em você e nos benefícios que essa mudança trará para sua vida. Você está no caminho certo para otimizar seu tempo e viver de forma mais equilibrada e feliz!


--------------------------------------------------------------

--- Aqui está seu relatório, espero que te ajude ---



> Olá! Entendo que você está buscando otimizar seu tempo e reduzir o uso do celular, especialmente em casa e com foco no YouTube, já tendo inclusive identificado o problema e querendo resolvê-lo! Abaixo, preparei um relatório com algumas informações e dicas que podem te ajudar nessa jornada.
> 
> **Resumo:**
> 
> O uso excessivo de celular pode impactar negativamente diversas áreas da vida, como hobbies, relacionamentos e saúde. Este relatório visa auxiliar na busca por um equilíbrio saudável, com foco na otimização do tempo e redução do uso do YouTube em casa.
> 
> **Dados Relevantes:**
> 
> *   **Tempo médio:** Usuários passam em média 5 horas por dia no celular.
> *   **YouTube:** Plataforma com vídeos feitos para serem viciantes.
> *   **Impacto em casa:** Redução do tempo com a família e outras atividades.
> *   **Recomendação:** Limitar o uso diário do celular a no máximo 2 horas.
> 
> **Estratégias Personalizadas:**
> 
> 1.  **Horários:** Defina horários específicos para usar o YouTube em casa (ex: 30 minutos pela manhã e à noite).
> 2.  **Aplicativos:** Utilize aplicativos para monitorar e bloquear o uso excessivo de determinados apps.
> 3.  **Notificações:** Desative as notificações do YouTube para evitar interrupções constantes.
> 4.  **Substituição:** Pratique seu hobby, andar de skate, ou encontre novas atividades prazerosas.
> 
> **Alternativas de Hobbies:**
> 
> *   Andar de Skate
> *   Esportes
> *   Leitura
> *   Música
> *   Cozinhar
> 
> **Benefícios de Reduzir o Uso:**
> 
> *   Mais tempo livre
> *   Melhor foco e produtividade
> *   Menos ansiedade e estresse
> *   Melhor qualidade do sono
> *   Mais conexão com pessoas próximas
> 
> **Mensagem de Encorajamento:**
> 
> Mudar hábitos exige tempo e persistência. Celebre cada pequena conquista e acredite no impacto positivo dessa mudança em sua vida.


--------------------------------------------------------------
